# Case analítico: Desafio de ciência de dados Lighthouse/Indicium  

# Considerações iniciais do projeto:

#### Objetivo:

Este relatório faz parte de um case analítico proposto como projeto teste para vaga de estágio na INDICIUM para a área de ciência de dados e tem como objetivo analisar um banco de dados cinematográfico(IMDB) para encontrar uma forma de orientar qual o tipo de filme que deve ser o próximo a entrar em fase de desenvolvimento.

#### Dicionário dos dados:

aqui estão as colunas que iremos trabalhar neste projeto e o significado de cada uma das variáveis nelas contidas.

| Coluna         | Descrição                                   |
|----------------|--------------------------------------------|
| Genre          | Gênero do filme                             |
| IMDB_Rating    | Nota do IMDB                                |
| Overview       | Overview do filme                           |
| Meta_score     | Média ponderada de todas as críticas       |
| Director       | Diretor do filme                            |
| Star1          | Ator/atriz principal #1                     |
| Star2          | Ator/atriz principal #2                     |
| Star3          | Ator/atriz principal #3                     |
| Star4          | Ator/atriz principal #4                     |
| No_of_Votes    | Número de votos                             |
| Gross          | Faturamento do filme                        |


## 1.0 Carregando bibliotecas que serão utilizadas na primeira parte do projeto(EDA):

In [131]:
#bibliotecas
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
from collections import Counter

#estilo adotado no seaborn para cores e estilização dos gráficos
sns.set(style="white")

#### 1.1 configurando o caminho do csv

In [63]:
#caminho para a pasta
path = "C:/Users/orfeu/Desktop/LH_DC_Project/data"

In [105]:
#leitura do dataset que será utilizado
df = pd.read_csv(os.path.join(path, 'desafio_indicium_imdb.csv'))

## 2.0 Exploração inicial dos dados:

In [107]:
display(df.head(3))

,Unnamed: 0,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"


In [113]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     999 non-null    int64  
 1   Series_Title   999 non-null    object 
 2   Released_Year  999 non-null    object 
 3   Certificate    898 non-null    object 
 4   Runtime        999 non-null    object 
 5   Genre          999 non-null    object 
 6   IMDB_Rating    999 non-null    float64
 7   Overview       999 non-null    object 
 8   Meta_score     842 non-null    float64
 9   Director       999 non-null    object 
 10  Star1          999 non-null    object 
 11  Star2          999 non-null    object 
 12  Star3          999 non-null    object 
 13  Star4          999 non-null    object 
 14  No_of_Votes    999 non-null    int64  
 15  Gross          830 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 125.0+ KB


None

In [111]:
display(df.describe())

,Unnamed: 0,IMDB_Rating,Meta_score,No_of_Votes
count,999.000000,999.000000,842.000000,9.990000e+02
mean,500.000000,7.947948,77.969121,2.716214e+05
std,288.530761,0.272290,12.383257,3.209126e+05
min,1.000000,7.600000,28.000000,2.508800e+04
25%,250.500000,7.700000,70.000000,5.547150e+04
50%,500.000000,7.900000,79.000000,1.383560e+05
75%,749.500000,8.100000,87.000000,3.731675e+05
max,999.000000,9.200000,100.000000,2.303232e+06


In [115]:
display(df.isnull().sum())

Unnamed: 0         0
Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

## 3.0 Tratamento inicial dos dados:

#### 3.1 Transformando o ano para número inteiro:

In [119]:
df["Released_Year"] = pd.to_numeric(df["Released_Year"], errors="coerce")

#### 3.2 pegar apenas o runtime(tempo de duração) dofilme:

In [123]:
df["Runtime_min"] = df["Runtime"].str.replace(" min", "").astype(float)

#### 3.3 convertendo a coluna de faturamento(gross) para numérica(remoção das vírgulas)

In [126]:
df["Gross"] = df["Gross"].str.replace(",", "")
df["Gross"] = pd.to_numeric(df["Gross"], errors="coerce")

#### 3.4 quebrando os generos de cada filme(as vezes um filme pode ter mais de um genero)

In [133]:
genres = df["Genre"].dropna().str.split(", ")
todos_generos = [g for lista in genres for g in lista]
contagem_generos = Counter(todos_generos)

#### 3.5 Criando uma coluna com a década em que o filme foi lançado:

In [145]:
# Aqui eu achei necessário pois pretendo fazer uma
# correlação da década que o filme foi lançado e seu faturamento,
# a nível de reconhecimento dos dados
df["Decada"] = (df["Released_Year"] // 10) * 10

## 4.0 Distribuição das variáveis numéricas:

#### 4.1 Notas do IMDB:

#### 4.2 Número de votos:

#### 4.3 Faturamento(Gross):

#### 4.4 Tempo de Duração(runtime):

## 5.0 Correlação entre as variáveis

#### 5.1

#### 5.2

#### 5.3

#### 5.4

## 6.0 Tendências temporais:

#### 6.1 filmes lançados por década:

#### 6.2 Evolução da nota média ao longo das décadas por gênero(?)

#### 6.3 Evolução do faturamento mediano ao longo das décadas por gênero(?)